# 题目

> 给定一组互不相同的单词，找出所有不同的索引对 (i, j) ，使得列表中的两个单词， words[i] + words[j] ，可拼接成回文串。

# 方法一：字典树

> 对于每一个字符串 $s$ ，都可以拆解成 $s_1$ 和 $s_2$ 两部分：  
其中一部分（假设是 $s_1$ ）是一个回文串（空字符串也是回文串），另一部分（假设是 $s_2$ ）是一个子字符串。  
那么可以在列表 words 中寻找是否有与 $s_2$ 的翻转相同的字符串，若有，则找到了一个答案。  

> 可以使用字典树储存所有单词，并在字典树上查找某个字符串的翻转是否存在。

## 复杂度

- 时间复杂度： $O(n×m^2)$，其中 $n$ 是字符串的数量， $m$ 是字符串的平均长度。

> 对于每一个字符串，我们需要 $O(m^2)$ 地判断其所有前缀与后缀是否是回文串，并 $O(m^2)$ 地寻找其所有前缀与后缀是否在给定的字符串序列中出现。

- 空间复杂度：$O(n×m)$，其中 $n$ 是字符串的数量， $m$ 是字符串的平均长度。

> 字典树的空间开销。

## 代码

In [1]:
# 定义字典树的节点
class Node:
    def __init__(self):
        self.ch = [0] * 26
        self.flag = -1

# 解法
class Solution:
    def palindromePairs(self, words):
        # 初始化一个树，包含一个根节点，根节点不表示任何字母
        tree = [Node()]

        # 字典树方法：插入一个字符串s
        def insert(s, index):
            length = len(s)
            add = 0  # 用于指示节点的位置
            
            # 遍历一个字符串，将这个字符串记录在字典树中
            # 该循环的效果如下：
            # 假设当前字符串为'abc'
            # i=0时，add=0，当前节点为tree[0]，即根节点。x=0，tree[0].ch[0]==0，因此添加一个子节点到列表尾部，该子节点代表位置[0]处的a
            # 该子节点在tree中的位置为[1]，因此令tree[0].ch[0]=1以指向这个子节点。并且令add=1，以便接下来的遍历。
            # i=1时，add=1，当前节点为tree[1]，即代表位置[0]处的a的子节点。x=1，tree[1].ch[1]==0，因此添加一个子节点到列表尾部，代表位置[1]处的b
            # 该子节点在tree中的位置为[2]，因此令tree[1].ch[1]=2以指向这个子节点。并且令add=2，以便接下来的遍历。
            # i=2时，add=2，进行类似操作。
            # 字符串'abc'遍历结束后，将代表字符串末尾字符的节点tree[3]的flag属性设置为index
            for i in range(length):
                x = ord(s[i]) - ord("a")  # 当前字母s[i]在ch中的位置
                # 若当前字符在字典树的对应位置不存在
                if tree[add].ch[x] == 0:
                    # 在tree列表尾部添加一个子节点用于表示当前字符
                    tree.append(Node())
                    # len(tree) - 1标示了一个子节点在tree列表中的位置，这个子节点代表当前的字符s[i]
                    tree[add].ch[x] = len(tree) - 1
                # 记录子节点在列表中的位置，for循环的下一个遍历时移动到该节点
                add = tree[add].ch[x]
            
            # 代表字符串末尾字符的节点的flag属性设为index已记录当前字符在列表words中的位置
            tree[add].flag = index
        
        # 输入字符串s和它的头尾位置left、right，返回s的翻转的位置
        def findWord(s, left, right):
            add = 0
            # 从右往左遍历逆序遍历s，查看s的翻转是否在字典树中
            for i in range(right, left - 1, -1):
                x = ord(s[i]) - ord("a")
                if tree[add].ch[x] == 0:
                    return -1
                add = tree[add].ch[x]
            # 若字典树中包含s的翻转，返回这个字符串的位置
            return tree[add].flag
        
        # 输入字符串s和它的头尾位置left、right，返回s是否为回文串
        def isPalindrome(s, left, right):
            length = right - left + 1
            # 分别从s的头尾遍历到中间，若每个字母都相等，则返回True
            return length < 0 or all(s[left + i] == s[right - i] for i in range(length // 2))
        
        n = len(words)
        # 将所有单词插入字典树
        for i, word in enumerate(words):
            insert(word, i)
        
        ret = list()
        # 遍历每个单词
        for i, word in enumerate(words):
            m = len(word)
            # 遍历0到m
            for j in range(m + 1):
                
                # 查看单词word从第j个字符开始的后缀是否是回文串
                if isPalindrome(word, j, m - 1):
                    # 若是，则寻找一个能接在word右边的单词
                    # 在字典树中查找word中以第j-1个字符为结尾的前缀的翻转，返回那个单词的位置
                    leftId = findWord(word, 0, j - 1)
                    # 如果那个单词存在且不是word本身
                    if leftId != -1 and leftId != i:
                        # 将那个单词接在word右边
                        ret.append([i, leftId])
                
                # 查看单词word以第j-1个字符为结尾的前缀是否是回文串
                if j and isPalindrome(word, 0, j - 1):
                    # 若是，则寻找一个能接在word左边的单词
                    # 在字典树中查找word中从第j个字符开始的后缀的翻转，返回那个单词的位置
                    rightId = findWord(word, j, m - 1)
                    # 如果那个单词存在且不是word本身
                    if rightId != -1 and rightId != i:
                        # 将那个单词接在word左边
                        ret.append([rightId, i])

        return ret

#### 测试一

In [2]:
words = ["abcd","dcba","lls","s","sssll"]

test = Solution()
test.palindromePairs(words)

[[0, 1], [1, 0], [3, 2], [2, 4]]

#### 测试二

In [3]:
words = ["a",""]

test = Solution()
test.palindromePairs(words)

[[0, 1], [1, 0]]